In [5]:
import pandas as pd

df = pd.read_csv('missionData_llm.csv')

In [6]:
df = df.replace('km', 'nm', regex=True)
df.to_csv('missionData_llm.csv', index=False)

In [13]:
print(df.head())

                                              prompt  \
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   

                                          completion  label  
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  


In [33]:
test_data_dict = {
    'prompt': df['prompt'].tolist()[0:4] + df['prompt'].tolist()[94:],
    'completion': df['completion'].tolist()[0:4] + df['completion'].tolist()[94:],
    'label': df['label'].tolist()[0:4] + df['label'].tolist()[94:]
}

train_data_dict = {
    'prompt': df['prompt'].tolist()[4:94], 
    'completion': df['completion'].tolist()[4:94], 
    'label': df['label'].tolist()[4:94]
}

In [34]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_data_dict)
test_dataset = Dataset.from_dict(test_data_dict)

dataset_dict = {
    'train': train_dataset,
    'test': test_dataset,
}

In [35]:
print(dataset_dict)

{'train': Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 90
}), 'test': Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 10
})}


In [37]:
!python kto.py \
    --per_device_train_batch_size 8 \
    --num_train_epochs 1 \
    --learning_rate 1e-4 \
    --lr_scheduler_type=cosine \
    --gradient_accumulation_steps 1 \
    --logging_steps 10 \
    --eval_steps 500 \
    --output_dir=./kto-aligned-model-lora \
    --warmup_ratio 0.1 \
    --report_to wandb \
    --bf16 \
    --logging_first_step \
    --use_peft \
    --load_in_4bit \
    --lora_target_modules=all-linear \
    --lora_r=16 \
    --lora_alpha=16

# python kto.py --per_device_train_batch_size 8 --num_train_epochs 1 --learning_rate 1e-4 --lr_scheduler_type=cosine --gradient_accumulation_steps 1 --logging_steps 10 --eval_steps 500 --output_dir=./kto-aligned-model-lora --warmup_ratio 0.1 --report_to wandb --bf16 --logging_first_step --use_peft --load_in_4bit --lora_target_modules=all-linear --lora_r=16 --lora_alpha=16


Loading checkpoint shards: 100%|##########| 4/4 [00:05<00:00,  1.30s/it]
